# Integrating ArcadeAI tools using OpenAI client

### Arcade-AI

In [ ]:
from arcadepy import Arcade
import os
from dotenv import load_dotenv
import logging
import json

load_dotenv()

arcade_client = Arcade()

In [ ]:
# Configure logging to display in the notebook
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create a stream handler to output logs to the notebook
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)

# Add a simple format to the handler
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(stream_handler)

In [ ]:
# Wrapper to execute ArcadeAI tools
def Google_CreateEvent(*args, **kwargs):

    response = arcade_client.tools.execute(
        tool_name="Google_CreateEvent",
        inputs=kwargs,
        user_id=os.environ['ARCADE_USER_ID'],
    )
    logger.info(response)
    return response

#### Convert Arcade tool to OpenAI compatible format

Steps to convert Arcade ToolDefinition to OpenAI schema:
  1. Convert Arcade ToolDefinition to a Pydantic Model.
  2. Transform the Pydantic Model into an OpenAI-compatible schema.

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_arcade._utilities import tool_definition_to_pydantic_model

def update_defaults(schema, defaults_list):
    """
    Update the default field of each key in the schema considering a list of dictionaries with keys to update.
    
    Args:
        schema (dict): The schema to update.
        defaults_list (list[dict]): A list of dictionaries with default values to update.
    """
    # Merge all dictionaries in the list into one
    merged_defaults = {}
    for default_dict in defaults_list:
        merged_defaults.update(default_dict)
    
    # Update the schema with the merged defaults
    for key, value in schema['function']['parameters']['properties'].items():
        if key in merged_defaults:
            value['default'] = merged_defaults[key]
    return schema

default_args = [
    {
        "attendee_emails": None
    }
]

def get_schema(tool_id, default_args):
    schema = convert_to_openai_tool(tool_definition_to_pydantic_model(arcade_client.tools.get(tool_id=tool_id)))
    schema = update_defaults(schema, default_args)
    schema['function']['name'] = tool_id
    logger.info(schema)
    return schema

schema = get_schema("Google_CreateEvent", default_args)

### OpenAI

In [ ]:
from openai import AsyncOpenAI

openai_client = AsyncOpenAI()

async def inference(input, tools):
    response = await openai_client.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": input},
            ],
            model="gpt-4o-mini",
            max_tokens=500,
            tools=tools,
            tool_choice='auto',
        )
    return response.choices[0].message


tools = [schema]

response = await inference("Calendar event for the date 2024-10-31 at 10:00 PM - 11:00 PM argentina timezone. The event is a meeting with Halloween party.", tools)

In [ ]:
if response.tool_calls:
    tool_call_id = response.tool_calls[0].id
    tool_function_name = response.tool_calls[0].function.name
    tool_query_string = json.loads(response.tool_calls[0].function.arguments)
    logger.info(globals()[tool_function_name](**tool_query_string))